#🚀 DAY 9 — Observability, Debugging & Evaluation for Agents

Today we will learn how real teams debug, monitor, and evaluate agents — not just “hope they work”.

**So far your agents can:**

   - ✔ plan
   - ✔ reason  
   - ✔ use tools
  - ✔ remember
  - ✔ access the web

**But now we answer:**

❓ How do I know my agent is correct, reliable, and improving?


**Build observable, debuggable, and evaluable agents by:**

- Logging thoughts, tool calls, and errors

- Tracing agent execution

- Evaluating correctness, reasoning quality, and tool usage

**🔹 1️⃣ CORE TOPICS**

**🔍 Observability**

- Execution traces

- Step-by-step logs

- Tool-call visibility

- Latency & failure points

**🧪 Evaluation**

- Correctness evaluation

- Reasoning quality

- Tool-choice accuracy

- Regression testing for agents

**🛠 Debugging**

- Prompt failures

- Tool misuse

- Infinite loops

- Hallucinations

**🔹 2️⃣ WHY THIS MATTERS**

Without observability:

❌ You don’t know why the agent failed

❌ You can’t improve prompts

❌ You can’t deploy safely

**With observability:**

✔ Faster debugging

✔ Measurable improvement

✔ Production readiness

This is what separates hobby agents from real applications.

**✔️ Step 1 — Enable Verbose Logging**

🧠 What is initialize_agent()?

Think of initialize_agent() as a factory function.

👉 It builds an agent for you automatically instead of you writing:

- prompts

- reasoning loops

- tool-selection logic

So instead of manually coding ReAct, LangChain does it internally.

**🧩 Line-by-Line Explanation**


1️⃣ tools

    tools = [add_numbers, ai_fact, web_search]
    #“These are the actions my agent is allowed to take.”

2️⃣ llm

    #“Use this LLM to think, reason, and decide.”
    llm = ChatOpenAI(model="gpt-4o-mini")

3️⃣ agent_type="zero-shot-react-description"

🔹 Zero-shot

The agent is not given examples.

It figures out:

how to reason

how to use tools

purely from descriptions.

🔹 ReAct

Reason + Act.

The agent:

Thinks

Decides to use a tool

Executes tool

Thinks again

Repeats


🔹 Description

The agent uses:

tool docstrings

function descriptions

to decide which tool to use.

    example:
    #Create a ReAct agent that can use tools by reading their descriptions, without examples

    @tool
    def add_numbers(a: int, b: int):
        """Add two numbers"""



4️⃣ verbose=True

🔹 What does verbose mean?

verbose=True means:

🔊 “Show me EVERYTHING the agent is thinking and doing.”

**🧠 Without verbose**

You see only:

    Final Answer: 15

**🧠 With verbose=True**

You see:

    Thought: I should calculate 5 + 10
    Action: add_numbers
    Action Input: {"a": 5, "b": 10}
    Observation: 15
    Thought: Now I can answer
    Final Answer: 15


In production → verbose=False
In learning → ALWAYS True


initialize_agent = "Build me a smart agent"

tools            = "Here are its
hands"

llm              = "Here is its
brain"

agent_type       = "How it should think"

verbose=True     = "Show me its thoughts"


In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent_type="zero-shot-react-description",
    verbose=True
)


**✔️ Step 2 — Add Structured Logging**

🔹 What Is This Code Doing (High-Level)?

This code is creating a simple logging system to track:

- what step the agent is on

- which tool it used

- what result it got

- when it happened

Think of it as a flight recorder for your agent ✈️


**🔹 Line-by-Line Explanation**

1️⃣ import time
Used for:

- ordering steps

- measuring duration

- debugging

      import time
      time.time()  # 1723456789.123

2️⃣ logs = []

- This list stores all agent actions.

- Each action will be added as a dictionary.

      logs = []

3️⃣ def log_step(...)
step → description of what happened

tool=None → optional argument (default is None)

result=None → optional argument

👉 You can call this function with:

just step or with tool and result


      def log_step(step, tool=None, result=None):


4️⃣ logs.append({ ... })

Each log entry is a dictionary.

    logs.append({
5️⃣ "timestamp": time.time()

Save when this step happened.

Later you can:

- sort logs

- calculate duration between steps

Example:

end - start = execution_time

    "timestamp": time.time(),


**🔹 Mental Model (Remember This)**

logs = agent diary
log_step() = write in diary
time.time() = when it happened

In [ ]:
import time

logs = []

def log_step(step, tool=None, result=None):
    logs.append({
        "timestamp": time.time(),
        "step": step,
        "tool": tool,
        "result": result
    })


**✔️ Step 3 — Trace Tool Calls**



In [ ]:
def execute_step(step):
    start = time.time()
    result = agent.run(step)
    log_step(step, tool="agent", result=result)
    print(f"Latency: {time.time() - start:.2f}s")
    return result


**🔹 5️⃣ AGENT EVALUATION (VERY IMPORTANT)**

In [ ]:
def evaluate(output, expected_keywords):
    score = sum(1 for k in expected_keywords if k.lower() in output.lower())
    return score / len(expected_keywords)


**✔️ Run Automated Tests**

In [ ]:
tests = [
    ("Explain RAG", ["retrieval", "generation"]),
    ("Add 5 and 10", ["15"])
]

for task, expected in tests:
    out = agent.run(task)
    print(task, evaluate(out, expected))
